<h1>Programmatically accessing 3DEP data using USGS 7.5' Quadrangles<h1>

<h2>Table of Contents<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Authors" data-toc-modified-id="Authors-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Authors</a></span></li><li><span><a href="#Purpose" data-toc-modified-id="Purpose-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Purpose</a></span></li><li><span><a href="#Funding" data-toc-modified-id="Funding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Funding</a></span></li><li><span><a href="#Keywords" data-toc-modified-id="Keywords-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Keywords</a></span></li><li><span><a href="#Citation" data-toc-modified-id="Citation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Citation</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Library-Imports" data-toc-modified-id="Library-Imports-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Library Imports</a></span><class="toc-item"><li><span><a href="#Library-Imports" data-toc-modified-id="Library-Imports-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Define Functions</a></span><class="toc-item"><li><span><a href="#Define-Functions" data-toc-modified-id="Define-Functions-6.2"></a></span></li><li><span><a href="#Data-Access-and-Visualization" data-toc-modified-id="Data-Access-and-Visualization-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Data Access and Visualization</a></span><ul class="toc-item"></ul></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Conclusion</a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Resources</a></span></li></ul></div>

<h2>Authors</h2>
<a id='#Authors'></a>

<h2>Purpose</h2>
<a id='#Purpose'></a>

The 3-D Elevation Program (3DEP), managed by the U.S. Geological Survey (USGS) National Geospatial Program, is an ongoing effort to provide high-quality topographic data for the entire conterminous United States. There are currently over 1800 3DEP datasets consisting of >37 trillion individual points and covering an area of > 6 million $km^{2}$.

While 3DEP data can be downloaded in the web browser from a variety of resources, it may benefit some users to programmatically access 3DEP point cloud data from a specific area of interest directly to their local directory. 3DEP data This notebook documents a workflow for accessing, processing, and visualizing data from the 3D Elevation Program (3DEP). The workflow capitalizes on available API and cloud resources allowing for highly compututationally expensive and intensive processes to be conducted and thereby not requiring high-end computational power from the user's local workstation.

A 7.5 minute quadrangle map covers an area of 49 to 70 square miles (130 to 180 km2). ###Worth thinking about how many points will be within each quad in this case.


This notebook includes two options for requesting 3DEP data using the USGS 7.5' Quadrangle footprints.

```Option 1```: The user inputs a USGS 7.5' quad ID (can be found here: https://www.usgs.gov/faqs/where-can-i-find-indexes-usgs-topographic-maps) and requests the data within the quad footprint.

```Option 2```: The user selects and area of interest (AOI) on an interactive map and requests the data within the quad(s) that overlay the user-defined AOI.

This notebook is one of three notebooks developed in an effort to increase ease of access for those who prefer to programmatically access 3DEP data. See the main Jupyter Notebook-based workflows for accessing 3DEP data. 

<h2>Funding</h2>
<a id='#Funding'></a>
OpenTopography is supported by the National Science Foundation under Award Numbers 1948997, 1948994 & 1948857. Funding for 3DEP workflow developement entitled "Enhancing usability of 3DEP data and web services with Jupyter notebooks"  was provided through the Community for Data Integration (CDI).

<h2>Keywords</h2>
<a id='#Keywords'></a>

keywords=["OpenTopography","USGS", "3DEP", "PDAL"]

<h2>Citation</h2>


<h3>Library Imports</h3>
<a id='#Library-Imports-6.1'></a>


In [2]:
import os
import json
import pdal
import numpy as np
import fiona
import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, GeoJSON, basemaps, basemap_to_tiles, projections
from osgeo import gdal
from shapely.geometry import shape, Point, Polygon
from shapely.ops import transform
import requests
import pyproj
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from tqdm import tqdm
import wget
import zipfile
import io

<h3>Define Functions</h3>
<a id='#Define-Functions-6.2'></a>

The functions below are used throughout the rest of the notebook and are included here for clarity.

```gcs_to_proj()``` is a function to project from geographic coordinates, WGS84 (EPSG: 4326), to Web Mercator projection (EPSG: 3857).

```handle_draw()``` is a function for interactive drawing on ipyleaflet maps and storing the polygon for use in requesting point cloud data.

```build_pdal_pipeline()``` is a function used to construct the pdal pipeline to request/process the point cloud data from AWS

These functions can be modified as the user sees fit; however, they are designed to work 'out of the box'.

In [3]:
def gcs_to_proj(poly):
    wgs84 = pyproj.CRS("EPSG:4326")
    web_mercator = pyproj.CRS("EPSG:3857")
    project = pyproj.Transformer.from_crs(wgs84, web_mercator, always_xy=True).transform
    user_poly_proj3857 = transform(project, poly)
    return(user_poly_proj3857)

def handle_draw(target, action, geo_json):
    geom = dict(geo_json['geometry'])
    user_poly = shape(geom)
    user_poly_proj3857 = gcs_to_proj(user_poly)
    print('AOI is valid and has boundaries of ', user_poly_proj3857.bounds)
    user_AOI.append((user_poly, user_poly_proj3857))  #for various reasons, we need user AOI in GCS and EPSG 3857

def build_pdal_pipeline(extent_epsg3857, usgs_3dep_dataset_name, resolution, outname = 'test.laz'):
    
    '''
    :param extent:
    :param extent_epsg:
    :param out_epsg:
    :param usgs_survey_name:
    :return:
    '''
   #reader = pdal.Reader.ept(url, polygon = AOI_EPSG3857_wtk, requests = 3)

    filename = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/{}/ept.json".format(usgs_3dep_dataset_name)
    
    ### we could allow the user to reproject from Web Mercator (EPSG 3857) to any coordinate system
    CRS = "EPSG:{}".format(out_epsg)

    build_pipeline = {"pipeline": [
        {
            "bounds": str(ent_extent),
            "filename": filename,
            "type": "readers.ept",
            "tag": "readdata"
        },
        {
            "limits": "Classification![7:7]",
            "type": "filters.range",
            "tag": "nonoise"
        }
        
        #outname
    ]}

    if not(pointResolution is None):
        build_pipeline['pipeline'][0]['resolution'] = pointResolution

#     build_pipeline = append_pointcloud_manipulation_pipeline(build_pipeline,extent,extent_epsg,out_epsg,filename,
#                                                              grid_name,doReclassify,doSavePointCloud)

    return build_pipeline

In [26]:
url = 'https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson'
r = requests.get(url)
with open('resources.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))

with open('resources.geojson', 'r') as f:
    data = json.load(f)

geo_json_3DEP = GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 
                                       'weight':1.9, 'fillOpacity':0.1})

with open('resources.geojson', 'r') as f:
    df = gpd.read_file(f)

projected_geoms = []

print('projecting 3DEP polygons to web mercator (EPSG:3857)')
for geometry in tqdm(df['geometry']):
        projected_geoms.append(gcs_to_proj(geometry))

df['geometry_epsg3857'] = projected_geoms
geometries_GCS_3DEP = df['geometry']
geometries_EPSG3857_3DEP = df['geometry_epsg3857']
names = df['name']
urls = df['url']
num_points = df['count']

projecting 3DEP polygons to web mercator (EPSG:3857)


100%|██████████████████████████████████████| 1807/1807 [00:08<00:00, 211.95it/s]
/Users/cole/anaconda3/envs/pointcloud/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [27]:
#download and unzip usgs 7.5 quadrangle footprints and save the files to a new directory called '\tmp'
if not os.path.exists('quad24.shp'):
    quad_url = 'https://mrdata.usgs.gov/reference/quad24.zip'
    local_path = 'tmp/'
    print("Downloading USGS 7.5' quadrangle footprint shapefile...")
    r = requests.get(quad_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path=local_path) # extract to folder called tmp
    filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'shp', 'shx'] if y.endswith(ending)] 
    print(filenames)

['quad24.dbf', 'quad24.shp', 'quad24.shx']


In [28]:
#make geopandas dataframe from the shapefile and inspect
dbf, shp, shx = [filename for filename in filenames]
quads = gpd.read_file(local_path + shp)
quads_ID = quads['QUAD24_ID']
geometries_GCS_quads = quads['geometry']
quads.head() #first 5 records in dataframe

,QUAD24_ID,Q24CODE,Q100COD,Q250COD,QUAD24NAME,QUAD24NAM2,QUAD100NAM,QUAD250NAM,STATE1,STATE2,STATE3,STATE4,X,Y,AREA,CORNER,CELL,geometry
0,1,49114NWF1,49114NW,49114,GOOSE LAKE,"GOOSE LAKE, MT",SAINT MARY,CUT BANK,MT,None,None,None,-113.3125,48.9375,1.273105e+08,NW,F1,"POLYGON ((-113.37500 49.00000, -113.25000 49.0..."
1,2,49114NWG1,49114NW,49114,HALL COULEE,"HALL COULEE, MT",SAINT MARY,CUT BANK,MT,None,None,None,-113.1875,48.9375,1.273067e+08,NW,G1,"POLYGON ((-113.25000 49.00000, -113.12500 49.0..."
2,3,49114NWH1,49114NW,49114,EMIGRANT GAP,"EMIGRANT GAP, MT",SAINT MARY,CUT BANK,MT,None,None,None,-113.0625,48.9375,1.273067e+08,NW,H1,"POLYGON ((-113.12500 49.00000, -113.00000 49.0..."
3,4,49114NEA1,49114NE,49114,BUSHNELL HILL,"BUSHNELL HILL, MT",CUT BANK,CUT BANK,MT,None,None,None,-112.9375,48.9375,1.273104e+08,NE,A1,"POLYGON ((-113.00000 49.00000, -112.87500 49.0..."
4,5,49114NEB1,49114NE,49114,DEL BONITA,"DEL BONITA, MT",CUT BANK,CUT BANK,MT,None,None,None,-112.8125,48.9375,1.273102e+08,NE,B1,"POLYGON ((-112.87500 49.00000, -112.75000 49.0..."


Note the column headings, specifically the ```QUAD24ID```, ```QUAD24NAME```, and ```geometry``` columns. The geometry is currently in geographic coordinate system WGS84 (EPSG: 4326). These will need to be reprojected to Web Mercator (EPSG: 3857), which is the projected coordinate system of the 3DEP EPT Tiles hosted in the Amazon bucket. Below the shapefile is converted to a GeoJSON format, and the reprojection happens in a subsequent cell. The below cell may raise a warning related to the pandas use of Int65Index, but it should not cause issues.

In [29]:
#convert gpd dataframe to GeoJSON format
quads.to_file('usgs_quadrangles.geojson', driver='GeoJSON')

with open('usgs_quadrangles.geojson', 'r') as f:
    data = json.load(f)
    
quads_geojson = GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 
                                       'weight':1.9, 'fillOpacity':0.1})

/Users/cole/anaconda3/envs/pointcloud/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


All cells up to this point are required for both ```Option 1``` and ```Option 2```. 
<br>Based on the User's desired output, proceed with either ```Option 1``` or ```Option 2``` below.

### Option 1 - User inputs USGS 7.5 Quadrangle ID (e.g., 35000)

Enter Quadrangle ID below. If the user does not know the QuadID for their area of interest, an interactive map to find the ID can be accessed here: https://www.usgs.gov/faqs/where-can-i-find-indexes-usgs-topographic-maps)

In [37]:
# Default QuadID 35000, which overlaps with an existing 3DEP datset. NOTE: Not all Quads overlap 3DEP data currently.
quad_ID = '47260'

In [38]:
# Required steps to get the geometry of the user-defined quad and make necessary projections
user_quad = quads.loc[quads['QUAD24_ID'] == int(quad_ID)]  # Find quad record in geodataframe 
user_quad_geom_GCS = user_quad['geometry'][user_quad.index[0]] # User-selected quad geometry in geographic coords
user_quad_GCS_wtk = user_quad_geom_GCS.wkt + f" / EPSG:4326" # Make WTK layer from geometry
user_quad_geom_EPSG3857 = gcs_to_proj(user_quad_geom_GCS)   # Reproject the footprint to EPSG:3857
user_quad_epsg3857_wtk = user_quad_geom_EPSG3857.wkt + f" / EPSG:3857" #Make WTK layer from projected geometry

Now the quadrangle footprint with the selected ```QUAD24_ID``` as well the 3DEP dataset polygons are added to an interactive ipyleaflet map for visual inspection.

In [39]:
quad_centroid =  list(user_quad_geom_GCS.centroid.coords)[0]

m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(quad_centroid[1], quad_centroid[0]),
    zoom=10,
    crs=projections.EPSG3857
    )

wlayer_quad = ipyleaflet.WKTLayer(
    wkt_string=user_quad_geom_GCS.boundary.wkt,
    style={"color": "blue"}
)

m.add_layer(geo_json_3DEP)  #add 3DEP polygons GeoJSON
m.add_layer(wlayer_quad)

#determine if your quad overlaps one of the existing 3DEP datasets
intersecting_polys = []
overlap = False
for i,geom in enumerate(geometries_EPSG3857_3DEP):
    if geom.intersects(user_quad_geom_EPSG3857):
        intersecting_polys.append((names[i], geometries_GCS_3DEP[i], geometries_EPSG3857_3DEP[i], urls[i], num_points[i]))
        overlap = True

if overlap == True:
    print("Quadrangle", quad_ID, "overlaps the USGS 3DEP dataset with name", intersecting_polys[0][0], 
          ". You may proceed.")
    
else:
    print("Quad", quad_ID, "doesn't overlap with any USGS 3DEP datasets at this time.")
m

Quadrangle 47260 overlaps the USGS 3DEP dataset with name USGS_LPC_TX_West_Central_B2_2018_LAS_2019 . You may proceed.


Map(center=[33.56249919495092, -101.9374997498912], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
url = intersecting_polys[0][3]
OUTPUT_RESOLUTION = 2.0
READ_RESOLUTION = 2.0

In [ ]:
### Reader ###
readers = []
reader = pdal.Reader.ept(url, polygon = user_quad_epsg3857_wtk, requests = 3)
readers.append(reader)

In [ ]:
writer = pdal.Writer.las(
    compression = "laszip",
    filename = "test.laz"
)

In [ ]:
pipeline = reader | writer

In [ ]:
%%time

# Use streaming mode at 1e6 points at a time. This
# helps us conserve memory for pipelines that are streamable
# check that with the pipeline.streamable property
results = pipeline.execute_streaming(chunk_size=1000000)
print(pipeline.log)


### Option 2: Use the user-defined AOI to find the intersecting quad(s) 

Make iPyLeaflet map for interactive drawing of user-defined AOI


In [33]:
m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(39, -100),
    zoom=3,
    crs=projections.EPSG3857
    )

m.add_layer(geo_json_3DEP)  #add 3DEP polygons GeoJSON

dc = ipyleaflet.DrawControl(
    marker={"shapeOptions": {"color": "#0000FF"}},
    rectangle={"shapeOptions": {"color": "#0000FF"}},
    circle={"shapeOptions": {"color": "#0000FF"}},
    circlemarker={},
)
### add to this map the quad polygons 
print("Select an Area of Interest where you would like USGS 7.5' quadrangle(s) using the tools on the left side of the map")
user_AOI = []
dc.on_draw(handle_draw)
m.add_control(dc)
m

Select an Area of Interest where you would like USGS 7.5' quadrangle(s) using the tools on the left side of the map


Map(center=[39, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

AOI is valid and has boundaries of  (-11348418.62341465, 3960477.827111507, -11340801.809896102, 3965914.767981327)


In [34]:
# Find intersecting quad(s) with the user AOI
AOI_GCS = user_AOI[0][0]
AOI_EPSG3857 = user_AOI[0][1]

intersecting_userAOI_quads = []
for i,geom in enumerate(geometries_GCS_quads):
    if geom.intersects(AOI_GCS):
        intersecting_userAOI_quads.append((quads_ID[i], geometries_GCS_quads[i]))

# Find intersecting 3DEP datasets with the quads
intersecting_quads_3DEP = []

for i, geom in enumerate(geometries_GCS_3DEP):
    for quad in intersecting_userAOI_quads:
        if geom.intersects(quad[1]):
            intersecting_quads_3DEP.append((names[i], geometries_GCS_3DEP[i], geometries_EPSG3857_3DEP[i],
                                            urls[i], num_points[i]))


In [63]:
area_quad = quads['AREA'][0]
area_3dep = intersecting_quads_3DEP[2][2].area
total_num_points = intersecting_quads_3DEP[2][4]
num_pts_est = int((area_quad/area_3dep)*(total_num_points))
num_pts_est

1307034361

In [40]:
intersecting_quads_3DEP

[('USGS_LPC_TX_West_Central_B2_2018_LAS_2019',
  'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_TX_West_Central_B2_2018_LAS_2019/ept.json',
  29302046820),
 ('USGS_LPC_TX_West_Central_B2_2018_LAS_2019',
  'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_TX_West_Central_B2_2018_LAS_2019/ept.json',
  29302046820),
 ('USGS_LPC_TX_West_Central_B5_2018',
  'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_TX_West_Central_B5_2018/ept.json',
  93951790440),
 ('USGS_LPC_TX_West_Central_B5_2018',
  'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_TX_West_Central_B5_2018/ept.json',
  93951790440)]

In [ ]:
### Map of the quads that intersect - Which would you like? Have hover-over for polygon selection

In [ ]:
### Construct pipeline 
